In [1]:
import pandas as pd
import numpy as np
from functions import *
import random
import csv
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier

remove_fight_island = False

In [2]:
#Load models
#REMINDER: We are going to need to use 'eval' to get the models usable
with open('models.csv', newline='') as f:
    reader = csv.reader(f)
    models = list(reader)
    
print(len(models))


###SELECT MODEL TO OPTIMIZE
model_num = 4

5


In [3]:
models

[['model_0',
  'model_1',
  'model_2',
  'model_3',
  'model_4',
  'model_06_13_20',
  'model_06_20_20',
  'model_6_27_20',
  'model_7_18_20',
  'model_7_25_20',
  'model_9_2_20',
  'model_9_10_20',
  'model_10_23_20',
  'model_10_27_20',
  'model_10_28_20'],
 ['LogisticRegression(max_iter=50000, random_state=75)',
  "DecisionTreeClassifier(criterion='entropy', max_depth=10, max_leaf_nodes=336,\n                       min_samples_leaf=0.01, random_state=75,\n                       splitter='random')",
  "RandomForestClassifier(criterion='entropy', max_depth=83, max_leaf_nodes=20,\n                       min_samples_leaf=0.009999000000000001, n_estimators=39,\n                       n_jobs=-1, random_state=75)",
  "GradientBoostingClassifier(max_depth=17, max_features='sqrt',\n                           max_leaf_nodes=316, min_samples_leaf=0.1,\n                           n_estimators=195, random_state=75)",
  'GaussianNB()',
  'LogisticRegression(max_iter=50000, random_state=75, tol=5.

In [4]:
df = pd.read_csv("../data/kaggle_data/ufc-master.csv")

In [5]:
#Let's put all the labels in a dataframe
df['label'] = ''
#If the winner is not Red or Blue we can remove it.
mask = df['Winner'] == 'Red'
df['label'][mask] = 0
mask = df['Winner'] == 'Blue'
df['label'][mask] = 1

#df["Winner"] = df["Winner"].astype('category')
df = df[(df['Winner'] == 'Blue') | (df['Winner'] == 'Red')]


#Make sure lable is numeric
df['label'] = pd.to_numeric(df['label'], errors='coerce')

<ipython-input-5-7548c2416a5f>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'][mask] = 0
<ipython-input-5-7548c2416a5f>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'][mask] = 1


In [6]:
#Let's fix the date
df['date'] = pd.to_datetime(df['date'])

In [7]:
#Create a label df:
label_df = df['label']

#Let's create an odds df too:
odds_df = df[['R_odds', 'B_odds']]

In [8]:
#Split the test set.  We are always(?) going to use the last 200 matches as the test set, so we don't want those around
#as we pick models

df_train = df[250:]
odds_train = odds_df[250:]
label_train = label_df[250:]

df_test = df[:250]
odds_test = odds_df[:250]
label_test = label_df[:250]

print(len(df_test))
print(len(odds_test))
print(len(label_test))

print(len(df_train))
print(len(odds_train))
print(len(label_train))

250
250
250
4509
4509
4509


In [9]:
if remove_fight_island:
    ##Let's remove the Fight island contests and see how that affects score
    df_test_no_fight_island = df_test[(df_test['location'] != 'Abu Dhabi, Abu Dhabi, United Arab Emirates')]
    df_train_no_fight_island = df_train[(df_train['location'] != 'Abu Dhabi, Abu Dhabi, United Arab Emirates')]
    df_test = df_test_no_fight_island
    df_train = df_train_no_fight_island



print(len(df_test))
print(len(df_train))


250
4509


In [10]:
display(df_train)
display(df_test)

,R_fighter,B_fighter,R_odds,B_odds,R_ev,B_ev,date,location,country,Winner,...,B_td_attempted_bout,R_td_pct_bout,B_td_pct_bout,R_sub_attempts_bout,B_sub_attempts_bout,R_pass_bout,B_pass_bout,R_rev_bout,B_rev_bout,label
250,Junior Dos Santos,Ciryl Gane,325,-400,325.000000,25.000000,2020-12-12,"Las Vegas, Nevada, USA",USA,Blue,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
251,Cub Swanson,Daniel Pineda,132,-152,132.000000,65.789474,2020-12-12,"Las Vegas, Nevada, USA",USA,Red,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
252,Renato Moicano,Rafael Fiziev,120,-140,120.000000,71.428571,2020-12-12,"Las Vegas, Nevada, USA",USA,Blue,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
253,Gavin Tucker,Billy Quarantillo,165,-190,165.000000,52.631579,2020-12-12,"Las Vegas, Nevada, USA",USA,Red,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
254,Tecia Torres,Sam Hughes,-550,425,18.181818,425.000000,2020-12-12,"Las Vegas, Nevada, USA",USA,Red,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4754,Duane Ludwig,Darren Elkins,-155,135,64.516129,135.000000,2010-03-21,"Broomfield, Colorado, USA",USA,Blue,...,2.0,0.00,0.5,0.0,0.0,0.0,0.0,0.0,0.0,1
4755,John Howard,Daniel Roberts,-210,175,47.619048,175.000000,2010-03-21,"Broomfield, Colorado, USA",USA,Red,...,1.0,1.00,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0
4756,Brendan Schaub,Chase Gormley,-260,220,38.461538,220.000000,2010-03-21,"Broomfield, Colorado, USA",USA,Red,...,2.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4757,Mike Pierce,Julio Paulino,-420,335,23.809524,335.000000,2010-03-21,"Broomfield, Colorado, USA",USA,Red,...,0.0,0.60,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0


,R_fighter,B_fighter,R_odds,B_odds,R_ev,B_ev,date,location,country,Winner,...,B_td_attempted_bout,R_td_pct_bout,B_td_pct_bout,R_sub_attempts_bout,B_sub_attempts_bout,R_pass_bout,B_pass_bout,R_rev_bout,B_rev_bout,label
0,Chan Sung Jung,Dan Ige,130,-162,130.000000,61.728395,2021-06-19,"Las Vegas, Nevada, USA",USA,Red,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,Aleksei Oleinik,Serghei Spivac,200,-265,200.000000,37.735849,2021-06-19,"Las Vegas, Nevada, USA",USA,Blue,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,Marlon Vera,Davey Grant,-230,180,43.478261,180.000000,2021-06-19,"Las Vegas, Nevada, USA",USA,Red,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,Julian Erosa,SeungWoo Choi,120,-150,120.000000,66.666667,2021-06-19,"Las Vegas, Nevada, USA",USA,Blue,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,Wellington Turman,Bruno Silva,100,-125,100.000000,80.000000,2021-06-19,"Las Vegas, Nevada, USA",USA,Blue,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,Jimmy Flick,Cody Durden,-157,137,63.694268,137.000000,2020-12-19,"Las Vegas, Nevada, USA",USA,Red,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
246,Christos Giagos,Carlton Minus,-345,285,28.985507,285.000000,2020-12-19,"Las Vegas, Nevada, USA",USA,Red,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
247,Tony Ferguson,Charles Oliveira,-185,160,54.054054,160.000000,2020-12-12,"Las Vegas, Nevada, USA",USA,Blue,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
248,Mackenzie Dern,Virna Jandiroba,-160,140,62.500000,140.000000,2020-12-12,"Las Vegas, Nevada, USA",USA,Red,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [11]:
#Set a value for the nulls in the ranks

weightclass_list = ['B_match_weightclass_rank', 'R_match_weightclass_rank', "R_Women's Flyweight_rank", "R_Women's Featherweight_rank", "R_Women's Strawweight_rank", "R_Women's Bantamweight_rank", 'R_Heavyweight_rank', 'R_Light Heavyweight_rank', 'R_Middleweight_rank', 'R_Welterweight_rank', 'R_Lightweight_rank', 'R_Featherweight_rank', 'R_Bantamweight_rank', 'R_Flyweight_rank', 'R_Pound-for-Pound_rank', "B_Women's Flyweight_rank", "B_Women's Featherweight_rank", "B_Women's Strawweight_rank", "B_Women's Bantamweight_rank", 'B_Heavyweight_rank', 'B_Light Heavyweight_rank', 'B_Middleweight_rank', 'B_Welterweight_rank', 'B_Lightweight_rank', 'B_Featherweight_rank', 'B_Bantamweight_rank', 'B_Flyweight_rank', 'B_Pound-for-Pound_rank']
df_train[weightclass_list] = df_train[weightclass_list].fillna(17)
df_test[weightclass_list] = df_test[weightclass_list].fillna(17)


C:\Users\matth\anaconda3\lib\site-packages\pandas\core\frame.py:3191: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [12]:
df_test
#df_test.to_csv('test.csv')

,R_fighter,B_fighter,R_odds,B_odds,R_ev,B_ev,date,location,country,Winner,...,B_td_attempted_bout,R_td_pct_bout,B_td_pct_bout,R_sub_attempts_bout,B_sub_attempts_bout,R_pass_bout,B_pass_bout,R_rev_bout,B_rev_bout,label
0,Chan Sung Jung,Dan Ige,130,-162,130.000000,61.728395,2021-06-19,"Las Vegas, Nevada, USA",USA,Red,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,Aleksei Oleinik,Serghei Spivac,200,-265,200.000000,37.735849,2021-06-19,"Las Vegas, Nevada, USA",USA,Blue,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,Marlon Vera,Davey Grant,-230,180,43.478261,180.000000,2021-06-19,"Las Vegas, Nevada, USA",USA,Red,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,Julian Erosa,SeungWoo Choi,120,-150,120.000000,66.666667,2021-06-19,"Las Vegas, Nevada, USA",USA,Blue,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,Wellington Turman,Bruno Silva,100,-125,100.000000,80.000000,2021-06-19,"Las Vegas, Nevada, USA",USA,Blue,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,Jimmy Flick,Cody Durden,-157,137,63.694268,137.000000,2020-12-19,"Las Vegas, Nevada, USA",USA,Red,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
246,Christos Giagos,Carlton Minus,-345,285,28.985507,285.000000,2020-12-19,"Las Vegas, Nevada, USA",USA,Red,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
247,Tony Ferguson,Charles Oliveira,-185,160,54.054054,160.000000,2020-12-12,"Las Vegas, Nevada, USA",USA,Blue,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
248,Mackenzie Dern,Virna Jandiroba,-160,140,62.500000,140.000000,2020-12-12,"Las Vegas, Nevada, USA",USA,Red,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [13]:
#1. Set features
#2. Set Hyperparameters
#3. Set EV
#4. Remove Features

In [14]:
models[1][model_num]

'GaussianNB()'

In [15]:
test_model_name = models[0][model_num]
test_model = eval(models[1][model_num])
test_model_features = eval(models[2][model_num])
test_model_ev = eval(models[3][model_num])

In [16]:
old_test_model = test_model
old_test_model_features = test_model_features
old_test_model_ev = test_model_ev

In [17]:
print(df_test.columns.tolist()) #Keeps us from truncating

['R_fighter', 'B_fighter', 'R_odds', 'B_odds', 'R_ev', 'B_ev', 'date', 'location', 'country', 'Winner', 'title_bout', 'weight_class', 'gender', 'no_of_rounds', 'B_current_lose_streak', 'B_current_win_streak', 'B_draw', 'B_avg_SIG_STR_landed', 'B_avg_SIG_STR_pct', 'B_avg_SUB_ATT', 'B_avg_TD_landed', 'B_avg_TD_pct', 'B_longest_win_streak', 'B_losses', 'B_total_rounds_fought', 'B_total_title_bouts', 'B_win_by_Decision_Majority', 'B_win_by_Decision_Split', 'B_win_by_Decision_Unanimous', 'B_win_by_KO/TKO', 'B_win_by_Submission', 'B_win_by_TKO_Doctor_Stoppage', 'B_wins', 'B_Stance', 'B_Height_cms', 'B_Reach_cms', 'B_Weight_lbs', 'R_current_lose_streak', 'R_current_win_streak', 'R_draw', 'R_avg_SIG_STR_landed', 'R_avg_SIG_STR_pct', 'R_avg_SUB_ATT', 'R_avg_TD_landed', 'R_avg_TD_pct', 'R_longest_win_streak', 'R_losses', 'R_total_rounds_fought', 'R_total_title_bouts', 'R_win_by_Decision_Majority', 'R_win_by_Decision_Split', 'R_win_by_Decision_Unanimous', 'R_win_by_KO/TKO', 'R_win_by_Submission',

In [18]:
#1. set features
my_pos_features = ['R_odds', 'B_odds', 'R_ev', 'B_ev',
       'location', 'country', 'title_bout', 'weight_class', 'gender',
       'no_of_rounds', 'B_current_lose_streak', 'B_current_win_streak',
       'B_draw', 'B_avg_SIG_STR_landed', 'B_avg_SIG_STR_pct', 'B_avg_SUB_ATT',
       'B_avg_TD_landed', 'B_avg_TD_pct', 'B_longest_win_streak', 'B_losses',
       'B_total_rounds_fought', 'B_total_title_bouts',
       'B_win_by_Decision_Majority', 'B_win_by_Decision_Split',
       'B_win_by_Decision_Unanimous', 'B_win_by_KO/TKO', 'B_win_by_Submission',
       'B_win_by_TKO_Doctor_Stoppage', 'B_wins', 'B_Stance', 'B_Height_cms',
       'B_Reach_cms', 'B_Weight_lbs', 'R_current_lose_streak',
       'R_current_win_streak', 'R_draw', 'R_avg_SIG_STR_landed',
       'R_avg_SIG_STR_pct', 'R_avg_SUB_ATT', 'R_avg_TD_landed', 'R_avg_TD_pct',
       'R_longest_win_streak', 'R_losses', 'R_total_rounds_fought',
       'R_total_title_bouts', 'R_win_by_Decision_Majority',
       'R_win_by_Decision_Split', 'R_win_by_Decision_Unanimous',
       'R_win_by_KO/TKO', 'R_win_by_Submission',
       'R_win_by_TKO_Doctor_Stoppage', 'R_wins', 'R_Stance', 'R_Height_cms',
       'R_Reach_cms', 'R_Weight_lbs', 'R_age', 'B_age', 'lose_streak_dif',
       'win_streak_dif', 'longest_win_streak_dif', 'win_dif', 'loss_dif',
       'total_round_dif', 'total_title_bout_dif', 'ko_dif', 'sub_dif',
       'height_dif', 'reach_dif', 'age_dif', 'sig_str_dif', 'avg_sub_att_dif',
       'avg_td_dif', 'empty_arena', 'B_match_weightclass_rank', 'R_match_weightclass_rank', 
        "R_Women's Flyweight_rank", "R_Women's Featherweight_rank", "R_Women's Strawweight_rank",
        "R_Women's Bantamweight_rank", 'R_Heavyweight_rank', 'R_Light Heavyweight_rank', 
        'R_Middleweight_rank', 'R_Welterweight_rank', 'R_Lightweight_rank', 'R_Featherweight_rank', 
        'R_Bantamweight_rank', 'R_Flyweight_rank', 'R_Pound-for-Pound_rank', "B_Women's Flyweight_rank", 
        "B_Women's Featherweight_rank", "B_Women's Strawweight_rank", "B_Women's Bantamweight_rank", 
        'B_Heavyweight_rank', 'B_Light Heavyweight_rank', 'B_Middleweight_rank', 'B_Welterweight_rank', 
        'B_Lightweight_rank', 'B_Featherweight_rank', 'B_Bantamweight_rank', 'B_Flyweight_rank', 
        'B_Pound-for-Pound_rank', 'better_rank']

In [19]:
print(test_model_name)
print(test_model)
print(test_model_features)
print(test_model_ev)



model_4
GaussianNB()
['R_longest_win_streak', 'longest_win_streak_dif', 'reach_dif', 'avg_sub_att_dif', 'R_win_by_Decision_Unanimous', 'B_win_by_Submission', 'R_Weight_lbs', 'sig_str_dif', 'B_Height_cms', 'B_avg_SUB_ATT', 'R_win_by_TKO_Doctor_Stoppage', 'height_dif', 'B_draw', 'avg_td_dif', 'R_win_by_Decision_Split', 'age_dif', 'B_current_win_streak', 'R_odds']
0


In [20]:
def save_model():
    score = evaluate_model(test_model, test_model_features, test_model_ev, df_train, label_train, odds_train, df_test, label_test,
                         odds_test, verbose = True)
    models[0][model_num] =  test_model_name 
    models[1][model_num] = test_model
    models[2][model_num] = test_model_features
    models[3][model_num] = test_model_ev
    models[4][model_num] = score    
    
    with open('models.csv', 'w', newline='') as outfile:
        writer = csv.writer(outfile)
        for row in models:
            print("HI")
            writer.writerow(row)

    outfile.close()    

In [21]:
def print_model():
    print()
    print(test_model_name)
    print(test_model)
    print(test_model_features)
    print(test_model_ev)
    print()

In [22]:
keep_going = True
#keep_going = False
while(keep_going):
    #1. Set Features
    #get_best_features(pos_features, m, df, cur_features, labels, odds, scale=False)
    test_model_features = (get_best_features(my_pos_features, test_model, df_train, test_model_features, label_train, odds_train, 
                                             min_ev=test_model_ev))
    print_model()
    save_model()
    #2 Set hyperparameters
    #def tune_hyperparameters(input_model, input_features, input_df, input_labels, odds_input):
    test_model = tune_hyperparameters(test_model, test_model_features, df_train, label_train, odds_train, 
                                      min_ev=test_model_ev)
    print_model()
    save_model()
    #3. Set EV
    #def tune_ev(input_model, input_features, input_df, input_labels, odds_input, verbose=False):
    test_model_ev = tune_ev(test_model, test_model_features, df_train, label_train, odds_train, verbose=False)

    old_test_model_features = test_model_features #This prevents
                                                  #an uneccesary loop
    print_model()
    save_model()
    #4. Remove Features
    #def remove_to_improve(cur_features, m, df, labels, odds, scale=False, min_ev = 0):
    test_model_features = remove_to_improve(test_model_features, test_model, df_train, label_train, odds_train, min_ev = test_model_ev)    
    keep_going = False
    if old_test_model != test_model:
        print("The hyperparameters are different")
        print("OLD:")
        print(old_test_model)
        print("NEW:")
        print(test_model)
        keep_going = True
        old_test_model = test_model
    if old_test_model_features != test_model_features:
        print("The features are different")
        print("OLD:")
        print(old_test_model_features)
        print("NEW:")
        print(test_model_features)
        keep_going = True
        old_test_model_features = test_model_features
    if old_test_model_ev != test_model_ev:
        print("The EV is different")
        print("OLD:")
        print(old_test_model_ev)
        print("NEW:")
        print(test_model_ev)
        keep_going = True
        old_test_model_ev = test_model_ev
    print_model()
    save_model()
    

    

Current best score is: 10417.633942622451
Feature: B_avg_SIG_STR_pct Score: 10481.027880415735
Feature: B_losses Score: 11722.814304152258
Feature: B_Stance Score: 14676.677243571316
Feature: B_Lightweight_rank Score: 16868.614152613227
The best feature was B_Lightweight_rank.  It scored 16868.614152613227
Current best score is: 16868.614152613227
Feature: B_current_lose_streak Score: 17336.40909112966
Feature: B_longest_win_streak Score: 17980.214900191724
Feature: B_losses Score: 18805.318279093044
Feature: R_age Score: 19303.353813657315
The best feature was R_age.  It scored 19303.353813657315
Current best score is: 19303.353813657315
Feature: B_longest_win_streak Score: 19669.25971829601
Feature: R_avg_TD_pct Score: 21927.37174363342
The best feature was R_avg_TD_pct.  It scored 21927.37174363342
Current best score is: 21927.37174363342
Feature: B_current_lose_streak Score: 22074.64420454246
Feature: R_total_rounds_fought Score: 23259.62786424984
The best feature was R_total_round

(3556, 23)

(3556,)

(3556, 2)

(250, 23)

...

(250, 23)

(250,)

(250, 2)

,t1_odds,t2_odds,t1_prob,t2_prob,winner
0,130,-162,0.536646,0.463354,0
1,200,-265,0.496289,0.503711,1
2,-230,180,0.615920,0.384080,0
3,120,-150,0.461141,0.538859,1
4,100,-125,0.476100,0.523900,1
...,...,...,...,...,...
245,-157,137,0.202560,0.797440,0
246,-345,285,0.870639,0.129361,0
247,-185,160,0.894885,0.105115,1
248,-160,140,0.661492,0.338508,0



          Number of matches: 250
          Number of bets: 231
          Number of winning bets: 129
          Number of losing bets: 102
          Number of underdog bets: 102
          Number of underdog wins: 46
          Number of underdog losses: 56
          Number of Favorite bets: 125
          Number of favorite wins: 82
          Number of favorite losses: 43
          Number of even bets: 4
          Number of even wins: 1
          Number of even losses: 3
          Profit: 1103.751826527694
          Profit per bet: 4.778146435184823
          Profit per match: 4.415007306110776
          
          
HI
HI
HI
HI
HI


In [23]:
#Get new score:
#def evaluate_model(input_model, input_features, input_ev, train_df, train_labels, train_odds, test_df, test_labels, test_odds, verbose=True):
score = evaluate_model(test_model, test_model_features, test_model_ev, df_train, label_train, odds_train, df_test, label_test,
                         odds_test, verbose = True)

(3556, 23)

(3556,)

(3556, 2)

(250, 23)

(250,)

(250, 2)

,t1_odds,t2_odds,t1_prob,t2_prob,winner
0,130,-162,0.536646,0.463354,0
1,200,-265,0.496289,0.503711,1
2,-230,180,0.615920,0.384080,0
3,120,-150,0.461141,0.538859,1
4,100,-125,0.476100,0.523900,1
...,...,...,...,...,...
245,-157,137,0.202560,0.797440,0
246,-345,285,0.870639,0.129361,0
247,-185,160,0.894885,0.105115,1
248,-160,140,0.661492,0.338508,0



          Number of matches: 250
          Number of bets: 231
          Number of winning bets: 129
          Number of losing bets: 102
          Number of underdog bets: 102
          Number of underdog wins: 46
          Number of underdog losses: 56
          Number of Favorite bets: 125
          Number of favorite wins: 82
          Number of favorite losses: 43
          Number of even bets: 4
          Number of even wins: 1
          Number of even losses: 3
          Profit: 1103.751826527694
          Profit per bet: 4.778146435184823
          Profit per match: 4.415007306110776
          
          


print(test_model_name) 
print(models[0][model_num])
print()
print(test_model)
print(eval(models[1][model_num]))
print()
print(test_model_features) 
print(eval(models[2][model_num]))
print()
print(test_model_ev)
print(eval(models[3][model_num]))

In [24]:
models[0][model_num] =  test_model_name 
models[1][model_num] = test_model
models[2][model_num] = test_model_features
models[3][model_num] = test_model_ev
models[4][model_num] = score

In [25]:
with open('models.csv', 'w', newline='') as outfile:
    writer = csv.writer(outfile)
    for row in models:
        print("HI")
        writer.writerow(row)
    
outfile.close()

HI
HI
HI
HI
HI


In [26]:
with open('models.csv', newline='') as f:
    reader = csv.reader(f)
    data = list(reader)
    
print(len(data))

5


In [27]:
data

[['model_0',
  'model_1',
  'model_2',
  'model_3',
  'model_4',
  'model_06_13_20',
  'model_06_20_20',
  'model_6_27_20',
  'model_7_18_20',
  'model_7_25_20',
  'model_9_2_20',
  'model_9_10_20',
  'model_10_23_20',
  'model_10_27_20',
  'model_10_28_20'],
 ['LogisticRegression(max_iter=50000, random_state=75)',
  "DecisionTreeClassifier(criterion='entropy', max_depth=10, max_leaf_nodes=336,\n                       min_samples_leaf=0.01, random_state=75,\n                       splitter='random')",
  "RandomForestClassifier(criterion='entropy', max_depth=83, max_leaf_nodes=20,\n                       min_samples_leaf=0.009999000000000001, n_estimators=39,\n                       n_jobs=-1, random_state=75)",
  "GradientBoostingClassifier(max_depth=17, max_features='sqrt',\n                           max_leaf_nodes=316, min_samples_leaf=0.1,\n                           n_estimators=195, random_state=75)",
  'GaussianNB()',
  'LogisticRegression(max_iter=50000, random_state=75, tol=5.